In [6]:
%load_ext autoreload
%autoreload 2

In [5]:
from main import read_file, get_sentences
import stanza

stanza.download("en", model_dir="../models/stanza_resources")

04/25/2023 17:51:38 - INFO - 	 Downloading default packages for language: en (English) ...
04/25/2023 17:51:40 - INFO - 	 File exists: ../models/stanza_resources/en/default.zip
04/25/2023 17:51:43 - INFO - 	 Finished downloading models and saved to ../models/stanza_resources.


In [1]:
henry_red = read_file("../data/english_short_stories/LeiningenVstheAnts.txt")

preprocess = stanza.Pipeline("en", processors="tokenize, ner, pos, lemma")

doc = preprocess(henry_red)

NameError: name 'read_file' is not defined

In [67]:
sentences = get_sentences(doc, lambda x: x.upos != "PUNCT")

In [83]:
from main import get_named_entities
named_entities = get_named_entities(doc)

names = []
for ne in named_entities:
    for named_entity in ne:
        names.append(named_entity.to_dict()[0]['lemma'])

from collections import Counter

name_frequencies = Counter(names)
name_frequencies = {k:v for (k,v) in sorted(name_frequencies.items(), key=lambda item: item[1], reverse=True)}
name_frequencies

{'Leiningen': 51,
 'Carl': 1,
 'Stephenson': 1,
 'Hood': 1,
 'Dullards': 1,
 'Pell': 1,
 '-': 1,
 'mell': 1,
 'Dawn': 1,
 'Macbeth': 1,
 'Birnam': 1,
 'Wood': 1,
 'Scot': 1,
 'Dante': 1,
 'Spry': 1,
 'Swift': 1}

In [85]:
import spacy
from fastcoref import spacy_component
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('fastcoref')

# This is better model
# nlp.add_pipe("fastcoref", config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': 'cpu'})

04/24/2023 21:48:53 - INFO - 	 missing_keys: []
04/24/2023 21:48:53 - INFO - 	 unexpected_keys: []
04/24/2023 21:48:53 - INFO - 	 mismatched_keys: []
04/24/2023 21:48:53 - INFO - 	 error_msgs: []
04/24/2023 21:48:53 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M


In [86]:
# henry_red = read_file("../data/english_short_stories/Henry_Red_Chief.txt")

# This substitutes pronouns with appropriate Named entities
coref_doc = nlp(henry_red, component_cfg={'fastcoref': {'resolve_text': True}})
resolved_text = coref_doc._.resolved_text

# Again perform NER on new text to get all the positions of the NE.
ne_doc = preprocess(resolved_text)
corrected_sentences = get_sentences(ne_doc, lambda x: x.upos != "PUNCT")
total_named_entities = get_named_entities(ne_doc)
total_names = []
for ne in total_named_entities:
    for named_entity in ne:
        total_names.append(named_entity.text)

from collections import Counter

total_name_frequencies = Counter(total_names)
total_name_frequencies = {k:v for (k,v) in sorted(total_name_frequencies.items(), key=lambda item: item[1], reverse=True)}

keys_to_delete = []
for name in total_name_frequencies:
    for name2 in total_name_frequencies:
        if name == name2[:-2]:
            total_name_frequencies[name] += total_name_frequencies[name2]
            keys_to_delete.append(name2)
        if name in name2 and name != name2:
            total_name_frequencies[name] += total_name_frequencies[name2]
            keys_to_delete.append(name2)
for k in keys_to_delete:
    if k in total_name_frequencies:
        total_name_frequencies.pop(k) 
total_name_frequencies


04/24/2023 21:48:58 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

04/24/2023 21:48:59 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

{'Leiningen': 303,
 'Macbeth': 3,
 'Carl': 1,
 'Stephenson': 1,
 'Hood': 1,
 'Pell': 1,
 '-': 1,
 'mell': 1,
 'Dawn': 1,
 'Birnam': 1,
 'Wood': 1,
 'Dante': 1,
 'Spry': 1,
 'Swift': 1}

In [9]:
from main import get_named_entities
from collections import Counter
import spacy
from fastcoref import spacy_component

def get_most_mentioned_characters(raw_text):
    # tokenize, pos, lemma, ner
    doc_ne = preprocess(raw_text)
    # extract only NE with PERSON tag
    named_entities = get_named_entities(doc_ne)

    # store all character lemmas
    names = []
    for ne in named_entities:
        for named_entity in ne:
            names.append(named_entity.to_dict()[0]['lemma'])

    # count frequencies of each name and sort them
    name_frequencies = Counter(names)
    name_frequencies = {k:v for (k,v) in sorted(name_frequencies.items(), key=lambda item: item[1], reverse=True)}

    # Coreference resolution
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe('fastcoref')

    # This substitutes pronouns with appropriate Named entities
    doc_coref = nlp(raw_text, component_cfg={'fastcoref': {'resolve_text': True}})
    resolved_text = doc_coref._.resolved_text

    # Again perform NER on new text to get all the positions of the NE.
    doc_ne_pronouns = preprocess(resolved_text)
    total_named_entities = get_named_entities(doc_ne_pronouns)
    total_names = []
    for ne in total_named_entities:
        for named_entity in ne:
            total_names.append(named_entity.to_dict()[0]['lemma'])

    # Again calculate frequencies...
    total_name_frequencies = Counter(total_names)
    total_name_frequencies = {k:v for (k,v) in sorted(total_name_frequencies.items(), key=lambda item: item[1], reverse=True)}

    # Join same names. E.g. Bill, Bill's...or Name Surname, Name
    keys_to_delete = []
    for name in total_name_frequencies:
        for name2 in total_name_frequencies:
            if name == name2[:-2]:
                total_name_frequencies[name] += total_name_frequencies[name2]
                keys_to_delete.append(name2)
            if name in name2 and name != name2:
                total_name_frequencies[name] += total_name_frequencies[name2]
                keys_to_delete.append(name2)
    for k in keys_to_delete:
        if k in total_name_frequencies:
            total_name_frequencies.pop(k)
    
    total_name_frequencies = {k:v for (k,v) in sorted(total_name_frequencies.items(), key=lambda item: item[1], reverse=True)}
    return total_name_frequencies
    


In [12]:
from main import read_file
import os

characters_by_stories = []

for f in os.listdir('../data/english_short_stories/'):
    raw_text = read_file(f'../data/english_short_stories/{f}')
    name_freqs = get_most_mentioned_characters(raw_text=raw_text)
    characters_by_stories.append(name_freqs)

Could not read file '../data/english_short_stories/Henry_Red_Chief.txt' using the utf-8 encoding
'utf-8' codec can't decode byte 0xd1 in position 123: invalid continuation byte
Could not read file '../data/english_short_stories/Henry_Red_Chief.txt' using the ascii encoding
'ascii' codec can't decode byte 0xd1 in position 123: ordinal not in range(128)
04/25/2023 17:56:57 - INFO - 	 missing_keys: []
04/25/2023 17:56:57 - INFO - 	 unexpected_keys: []
04/25/2023 17:56:57 - INFO - 	 mismatched_keys: []
04/25/2023 17:56:57 - INFO - 	 error_msgs: []
04/25/2023 17:56:57 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
04/25/2023 17:56:58 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

04/25/2023 17:56:58 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

04/25/2023 17:57:32 - INFO - 	 missing_keys: []
04/25/2023 17:57:32 - INFO - 	 unexpected_keys: []
04/25/2023 17:57:32 - INFO - 	 mismatched_keys: []
04/25/2023 17:57:32 - INFO - 	 error_msgs: []
04/25/2023 17:57:32 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
04/25/2023 17:57:32 - INFO - 	 Tokenize 1 inputs...


...

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

04/25/2023 18:00:12 - INFO - 	 missing_keys: []
04/25/2023 18:00:12 - INFO - 	 unexpected_keys: []
04/25/2023 18:00:12 - INFO - 	 mismatched_keys: []
04/25/2023 18:00:12 - INFO - 	 error_msgs: []
04/25/2023 18:00:12 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
04/25/2023 18:00:14 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

04/25/2023 18:00:15 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
import json
json_data = json.dumps(characters_by_stories, indent=4)
with open('../data/characters_by_stories.json', 'w') as outfile:
    outfile.write(json_data)